In [1]:
%load_ext pycodestyle_magic

In [2]:
%flake8_on

In [3]:
import numpy as np
from functools import reduce

In [4]:
testdata = """class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,4,50
55,2,20
38,6,12""".splitlines()

testdata2 = """class: 0-1 or 4-19
row: 0-5 or 8-19
seat: 0-13 or 16-19

your ticket:
11,12,13

nearby tickets:
3,9,18
15,1,5
5,14,9""".splitlines()

In [5]:
def parse_data(data):
    fields = dict()
    my_ticket = []
    nearby_tickets = []

    section = 1
    for line in data:
        if line == '':
            section += 1
            continue

        if line[-1] == ':':
            continue

        if section == 1:
            field, ranges = line.split(':')
            range1, range2 = ranges.split(' or ')
            range1_low, range1_high = range1.split('-')
            range2_low, range2_high = range2.split('-')

            fields[field] = [(int(range1_low), int(range1_high)),
                             (int(range2_low), int(range2_high))]
        elif section == 2:
            my_ticket = [int(p) for p in line.split(',')]
        elif section == 3:
            nearby_tickets.append([int(p) for p in line.split(',')])

    return fields, my_ticket, nearby_tickets

In [6]:
parse_data(testdata)

({'class': [(1, 3), (5, 7)],
  'row': [(6, 11), (33, 44)],
  'seat': [(13, 40), (45, 50)]},
 [7, 1, 14],
 [[7, 3, 47], [40, 4, 50], [55, 2, 20], [38, 6, 12]])

In [7]:
def validate(ticket, fields):
    error = 0
    valids = []
    for parameter in ticket:
        valid = validate_parameter(parameter, fields)
        if len(valid) == 0:
            error += parameter
        valids.append(valid)
    return error, valids, ticket


def validate_parameter(parameter, fields):
    valid = set()
    for field, ranges in fields.items():
        if (ranges[0][0] <= parameter <= ranges[0][1]
                or ranges[1][0] <= parameter <= ranges[1][1]):
            valid.add(field)
    return valid

In [8]:
fields, my_ticket, nearby_tickets = parse_data(testdata)
print(fields, my_ticket, nearby_tickets)
print(sum([validate(ticket, fields)[0] for ticket in nearby_tickets]))

{'class': [(1, 3), (5, 7)], 'row': [(6, 11), (33, 44)], 'seat': [(13, 40), (45, 50)]} [7, 1, 14] [[7, 3, 47], [40, 4, 50], [55, 2, 20], [38, 6, 12]]
71


In [9]:
with open('input', 'r') as inp:
    inputdata = [line.strip() for line in inp.readlines()]

fields, my_ticket, nearby_tickets = parse_data(inputdata)
# print(fields, my_ticket, nearby_tickets)
validations = [validate(ticket, fields) for ticket in nearby_tickets]

print(sum([error for error, _, _ in validations]))

valid = [(fields, ticket)
         for error, fields, ticket in validations
         if error == 0]

solutions = [(0, set())] * len(my_ticket)
for p in range(len(my_ticket)):
    fieldss_p = [fields[p] for fields, _ in valid]
    fieldss_p_possible = reduce(lambda a, b: a & b, fieldss_p)
    solutions[p] = (len(fieldss_p_possible), fieldss_p_possible)
# print(f'solutions: {solutions}')

solutions_ordered = [(0, set())] * len(my_ticket)
for p, (options, fs) in enumerate(solutions):
    solutions_ordered[options - 1] = (p, fs)
# print(f'solutions ordered: {solutions_ordered}')

eliminated = set()
results = [''] * len(my_ticket)
for p, fields in solutions_ordered:
    fields -= eliminated
    if len(fields) > 1:
        print('I can not do this!')
        break
    eliminated = eliminated | fields
    results[p] = fields.pop()
    # print(f'Field {p} must be {results[p]}')

print('Thus my ticket reads:')
for i, value in enumerate(my_ticket):
    print(f'  {results[i]}: {value}')

sought_idx = [i for i, field
              in enumerate(results) if field.startswith('departure')]
print(np.prod(np.array(my_ticket)[sought_idx]))

24980
Thus my ticket reads:
  arrival track: 79
  duration: 149
  departure time: 97
  departure station: 163
  class: 59
  type: 151
  departure date: 101
  wagon: 89
  arrival platform: 173
  price: 139
  arrival location: 167
  row: 61
  departure platform: 73
  zone: 71
  arrival station: 137
  departure location: 53
  train: 83
  route: 157
  departure track: 131
  seat: 67
809376774329
